In [ ]:
import os
from pathlib import Path
from typing import cast

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
from IPython.display import display

import modfs.utils.plotting as plot
from modfs.data.result_load import add_derived_columns, load_results

ROOT_PATH = Path("../../../").resolve()
FIGS_DIR = ROOT_PATH / "data/figs/paper/modular_scheduling"
FIGS_DIR.mkdir(exist_ok=True, parents=True)

results_path = ROOT_PATH / "data/run"

df_original = load_results(results_path)

df_derived = add_derived_columns(
    df_original,
    groups={
        "homogeneous": {"generic/printer_cases"},
        "heterogeneous": {"generic/mixed"},
        "computational_all": {"computational"},
    },
    gen_subpath="data/gen",
    group_source="run_file_path",
    run_subpath=results_path,
    exclude_paths=[],
)

# Set computational group for:
# - The constraint modular solver
# - The BHCS algorithm (whether it uses broadcast or cocktail)
# - The simple algorithm
selection = df_derived.eval(
    "group == 'computational_all' & "
    + "(modular_algorithm == 'constraint' | algorithm == 'bhcs' | algorithm == 'simple')"
)
df_derived.loc[selection, "group"] = "computational"  # type: ignore

# Set mixed group for the BHCS with MNEH and for MNEH only. We only care for results that use
# MNEH with the ASAP backtrack seeder algorithm.
selection = df_derived.eval(
    "group == 'computational_all' "
    + "& algorithm.isin(['bhcs-mneh-asap-backtrack', 'mneh-asap-backtrack'])"
)
df_derived.loc[selection, "group"] = "mixed"  # type: ignore

# Copy the results of BHCS and the constraint solver from the computational group to the mixed group
df_bhcs_comp = df_derived.query(
    "group == 'computational' & "
    + "((algorithm == 'bhcs' & time_limit == 3600) | modular_algorithm == 'constraint')"
).copy()
df_bhcs_comp["group"] = "mixed"
df_derived = pd.concat([df_derived, df_bhcs_comp], ignore_index=True)

df_all = df_derived.query(
    "group.isin(['homogeneous', 'heterogeneous', 'computational', 'mixed'])"
).copy()

# Set the names of the algorithms
df_all.loc[df_all.eval("modular_algorithm == 'constraint'"), "algorithm_name"] = "MON"  # type: ignore
df_all.loc[df_all.eval("modular_algorithm == 'cocktail'"), "algorithm_name"] = "MAS-C"  # type: ignore
df_all.loc[df_all.eval("modular_algorithm == 'broadcast'"), "algorithm_name"] = "MAS-B"  # type: ignore
df_all.loc[
    df_all.eval("modular_algorithm == 'cocktail' & algorithm == 'simple'"), "algorithm_name"
] = "MAS-CS"  # type: ignore
df_all.loc[
    df_all.eval("modular_algorithm == 'broadcast' & algorithm == 'simple'"), "algorithm_name"
] = "MAS-BS"  # type: ignore
df_all.loc[df_all.eval("group == 'mixed' & algorithm == 'bhcs'"), "algorithm_name"] = "MAS-BHCS"  # type: ignore
df_all.loc[
    df_all.eval("group == 'mixed' & algorithm == 'bhcs-mneh-asap-backtrack'"), "algorithm_name"
] = "MAS-INTL"  # type: ignore
df_all.loc[
    df_all.eval("group == 'mixed' & algorithm == 'mneh-asap-backtrack'"), "algorithm_name"
] = "MAS-MNEH"  # type: ignore
df_all.replace({"error": {"": "none"}}, inplace=True)

# Add the mixed_100 and mixed_80 groups
df_mixed_100 = df_all.query(
    "group == 'mixed' & modular_algorithm != 'broadcast' & jobs <= 100"
).copy()
df_mixed_100["group"] = "mixed_100"
df_mixed_60 = df_all.query(
    "group == 'mixed' & modular_algorithm != 'broadcast' & jobs == 60 & modules <= 6"
).copy()
df_mixed_60["group"] = "mixed_60"

df_all = pd.concat([df_all, df_mixed_100, df_mixed_60], ignore_index=True)

df_generic = df_all.query("group.isin(['homogeneous', 'heterogeneous'])")
df_computational = df_all.query(
    "group == 'computational' & ~(modular_algorithm == 'constraint' & time_limit == 600)"
)


del df_original, df_derived

figs = {}

In [ ]:
colors = {
    "notimeout": "tab:blue",
    "MON": "tab:green",
    "MAS": "tab:olive",
    "MAS-C": "tab:olive",
    "MAS-B": "tab:orange",
    "MAS-BS": "tab:purple",
    "MAS-CS": "tab:cyan",
    "MAS-BHCS": "tab:olive",
    "MAS-MNEH": "tab:purple",
    "MAS-INTL": "tab:cyan",
    600: "tab:purple",
    3600: "tab:cyan",
}
markers = {"MON": ".", "MAS": "*", "MAS-C": "*", "MAS-B": "o"}
hatches = {
    "MON": "",
    "MAS-C": "////",
    "MAS-B": r"\\\\",
    "MAS-BHCS": "////",
    "MAS-MNEH": r"---",
    "MAS-INTL": r"xxx",
    600: "////",
    3600: r"\\\\",
}

number_colors = {
    2: "tab:blue",
    3: "tab:orange",
    4: "tab:green",
    5: "tab:red",
    6: "tab:purple",
    7: "tab:brown",
    8: "tab:pink",
    9: "tab:gray",
    10: "tab:olive",
}

number_markers = {
    2: ".",
    3: "1",
    4: "2",
    5: "3",
    6: "4",
    7: "*",
    8: "+",
    9: "x",
    10: 8,
}

In [ ]:
df_all.query("modular_algorithm == 'constraint'")["solved"].mean()

In [ ]:
# We create a table with all the results, then we filter it to get the relevant results
# depending on the group or section that we want to analyze.
table = df_all.pivot_table(
    index=["group", "modular_algorithm", "algorithm", "time_limit"],
    columns="error",
    aggfunc="size",
    fill_value=0,
)
important_cols = table.columns.copy()
table["total"] = table.sum(axis=1)

table2 = table.drop(columns=["total"])
table2[important_cols] = table2[important_cols].div(table["total"], axis=0) * 100

table = pd.concat({"sum": table, "%": table2}, axis=1)
s = table.style
s.format("{:.1f}", ["%"])
s.set_table_styles([{"selector": "th", "props": "text-align: center;"}])
display(s)

######## GENERIC ########

# Take only heterogeneous and homogeneous
table_generic = cast(
    pd.DataFrame,
    table.query(
        "group in ['homogeneous', 'heterogeneous'] & modular_algorithm != 'constraint' "
        "& time_limit == 600 & algorithm != 'simple'"
    )["%"],
)
# Drop the last two index levels
table_generic.index = table_generic.index.droplevel([2, 3])
table_generic = table_generic.reindex(columns=["none", "local-scheduler", "time-out"])
rename_dict = {"local-scheduler": "Local", "none": "Solved", "time-out": "Time"}
table_generic = table_generic.rename(columns=rename_dict).T

display(table_generic)

plt.figure(figsize=(2.3, 2.3), dpi=250)
sns.set_theme(context="paper", style="whitegrid")
g = sns.heatmap(
    table_generic,
    annot=True,
    fmt=".1f",
    cmap="Blues",
    cbar=False,
    xticklabels=["MAS-B", "MAS-C"] * 2,
)
plt.ylabel("Termination type [%]")
plt.xlabel("Strategy")
g.tick_params(axis="both", labelsize="small")
g.tick_params(axis="x", rotation=0)

ax2 = g.twiny()
ax2.set_xticks(
    [1, 3],
    ["Heterogeneous", "Homogeneous"],
    # rotation="vertical",
    rotation_mode="default",
    fontsize="small",
    verticalalignment="center",
)
ax2.set_xlim(0, 4)
ax2.set_xlabel("Benchmark")
ax2.vlines([2], *ax2.get_ylim(), color="black", linewidth=1)
ax2.grid(False)

plt.savefig(FIGS_DIR / "termination_type_generic.pdf", bbox_inches="tight", pad_inches=0)
plt.show()


######## COMPUTATIONAL ########

table_comp = cast(
    pd.DataFrame,
    table.query(
        "group == 'computational' & modular_algorithm != 'constraint' & algorithm != 'simple'"
    )["%"],
)

table_comp.index = table_comp.index.droplevel([0, 2])
table_comp = table_comp.reindex(columns=["none", "local-scheduler", "time-out"])
table_comp = table_comp.rename(columns=rename_dict).T
display(table_comp)

plt.figure(figsize=(2.3, 2.3), dpi=250)
sns.set_theme(context="paper", style="whitegrid")
g = sns.heatmap(
    table_comp,
    annot=True,
    fmt=".1f",
    cmap="Blues",
    cbar=False,
    xticklabels=["600", "3600"] * 2,
)
plt.ylabel("Termination type [%]")
plt.xlabel("Strategy")
g.tick_params(axis="both", labelsize="small")
g.set_xticks([1, 3], ["MAS-B", "MAS-C"], rotation=0)

ax2 = g.twiny()
ax2.set_xticks(
    [0.5, 1.5, 2.5, 3.5],
    [600, 3600] * 2,
    fontsize="small",
    verticalalignment="center",
)
ax2.set_xlim(0, 4)
ax2.set_xlabel("Time limit [s]")
ax2.vlines([2], *ax2.get_ylim(), color="black", linewidth=1)
ax2.grid(False)

plt.savefig(FIGS_DIR / "termination_type_computational.pdf", bbox_inches="tight", pad_inches=0)
plt.show()

In [ ]:
# Draw boxplot comparing the makespan
df_other, df_baseline = plot.baseline_extract(df_all, "constraint", "modular_algorithm")


df_compared = plot.baseline_compare(df_generic, "constraint", "makespan")
display(df_compared.groupby(["group", "modular_algorithm"])["makespan"].mean().to_frame())
display(df_compared.groupby(["modular_algorithm"])["makespan"].mean().to_frame())

plt.figure(figsize=(2.2, 2.3), dpi=250)
sns.set_theme(context="paper", style="whitegrid")
g = sns.boxplot(
    data=df_compared,
    x="group",
    y="makespan",
    hue="algorithm_name",
    hue_order=["MAS-B", "MAS-C"],
    palette=colors,
    legend=False,
    whis=(5, 95),
    showfliers=False,
)

hatches_lst = [hatches["MAS-B"]] * 2 + [hatches["MAS-C"]] * 2

for patch, hatch in zip(g.patches, hatches_lst):
    patch.set_hatch(hatch)

plt.axhline(y=1, color="r")
plt.text(0.9, 0.93, "Baseline")

from matplotlib.patches import Patch

plt.legend(
    handles=[
        Patch(facecolor=colors["MAS-B"], edgecolor="k", hatch=hatches["MAS-B"], label="MAS-B"),
        Patch(facecolor=colors["MAS-C"], edgecolor="k", hatch=hatches["MAS-C"], label="MAS-C"),
    ],
    loc="upper right",
)

plt.ylabel("Relative makespan")
plt.xlabel("Benchmark")
plt.xticks([0, 1], ["Heterogeneous", "Homogeneous"])

plt.savefig(FIGS_DIR / "makespan_all.pdf", bbox_inches="tight", pad_inches=0)
plt.show()

In [ ]:
# Draw bar plot comparing number of finished solutions
df_completed = df_generic.groupby(["group", "modular_algorithm"])["solved"].mean()
display(df_completed.to_frame())

plt.figure(figsize=(2.2, 2.3), dpi=250)
sns.set_theme(context="paper", style="whitegrid")

xposition = [0, 0.4, 1, 1.4]
yposition = cast(
    list[float],
    [
        df_completed["heterogeneous", "broadcast"],
        df_completed["heterogeneous", "cocktail"],
        df_completed["homogeneous", "broadcast"],
        df_completed["homogeneous", "cocktail"],
    ],
)

plt.bar(
    xposition,
    yposition,
    width=0.4,
    edgecolor="k",
    color=[colors["MAS-B"], colors["MAS-C"]] * 2,
    hatch=[hatches["MAS-B"], hatches["MAS-C"]] * 2,
)
from matplotlib.patches import Patch

for x, y in zip(xposition, yposition):
    plt.text(x, y + 0.015, f"{y:.2f}", fontsize="small", horizontalalignment="center")

plt.legend(
    handles=[
        Patch(facecolor=colors["MAS-B"], edgecolor="k", hatch="\\\\\\\\", label="MAS-B"),
        Patch(facecolor=colors["MAS-C"], edgecolor="k", hatch="////", label="MAS-C"),
    ],
    loc="lower right",
)
plt.xticks([0.2, 1.2], ["Heterogeneous", "Homogeneous"])
plt.ylabel("Schedulability ratio")
plt.xlabel("Benchmark")
plt.ylim(0, 1)
plt.grid(axis="x")
plt.savefig(FIGS_DIR / "schedulability_ratio_all.pdf", bbox_inches="tight", pad_inches=0)
plt.show()

In [ ]:
# Draw boxplot comparing the total execution time
df_time = df_generic.copy()
df_time["total_time"] /= 1000

display(df_time.groupby(["group", "modular_algorithm"])["total_time"].mean().to_frame())

plt.figure(figsize=(2.2, 2.3), dpi=250)
sns.set_theme(context="paper", style="whitegrid")
g = sns.boxplot(
    data=df_time,
    x="group",
    y="total_time",
    hue="algorithm_name",
    hue_order=["MAS-B", "MAS-C"],
    palette=colors,
    legend=False,
    whis=(5, 95),
    showfliers=False,
    order=["heterogeneous", "homogeneous"],
)

hatches_lst = [hatches["MAS-B"]] * 2 + [hatches["MAS-C"]] * 2

for patch, hatch in zip(g.patches, hatches_lst):
    patch.set_hatch(hatch)

from matplotlib.patches import Patch

plt.legend(
    handles=[
        Patch(facecolor=colors["MAS-B"], edgecolor="k", hatch=hatches["MAS-B"], label="MAS-B"),
        Patch(facecolor=colors["MAS-C"], edgecolor="k", hatch=hatches["MAS-C"], label="MAS-C"),
    ],
    loc="right",
)

plt.xticks([0, 1], ["Heterogeneous", "Homogeneous"])
plt.ylabel("Average execution time [s]")
plt.xlabel("Benchmark")
plt.grid(axis="x")
plt.savefig(FIGS_DIR / "execution_time_all.pdf", bbox_inches="tight", pad_inches=0)
plt.show()

In [ ]:
from matplotlib.lines import Line2D
from matplotlib.patches import Rectangle


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 2.1), dpi=250)
sns.set_theme(context="paper", style="whitegrid")

ax1.axhline(1, color="r")
ax1.text(90, 0.95, "Baseline")

df_compared = plot.baseline_compare(df_generic, "constraint", "makespan")
for group, df in df_compared.query("modular_algorithm == 'cocktail' & algorithm == 'bhcs'").groupby(
    "modules"
):
    group = cast(int, group)
    # if group % 2 != 0:
    #     continue

    df_tmp = df.groupby("jobs", as_index=False)["makespan"].mean()
    ax1.plot(
        df_tmp["jobs"],
        df_tmp["makespan"],
        color=number_colors[group],
        marker=number_markers[group],
    )

l = ax1.legend(
    [
        Line2D(
            [],
            [],
            color=number_colors[i],
            marker=number_markers[i],
            markersize=5.0,
        )
        for i in range(2, 11, 1)
    ],
    [f"{i}" for i in range(2, 11, 1)],
    loc="upper left",
    bbox_to_anchor=(-0.023, 1.2),
    title="Modules",
    ncol=9,
    handlelength=1.85,
    columnspacing=0.8,
    handletextpad=0.6,
)


# From https://stackoverflow.com/a/53329898/4005637
c = l.get_children()[0]
title = c.get_children()[0]
hpack = c.get_children()[1]
c._children = [hpack]
hpack._children = [title] + hpack.get_children()

ax1.set_xlim(0, 305)
# ax1.set_ylim(0.9, 1.45)
ax1.set_ylabel("Relative makespan")
ax1.set_xlabel("Number of jobs [MAS-C]")


for group, df in df_generic.query("solved & algorithm == 'bhcs'").groupby("modules"):
    group = cast(int, group)
    df_tmp = df.groupby(["modular_algorithm", "jobs"])["iterations"].mean()
    ax2.plot(
        df_tmp.loc["cocktail"].index,
        df_tmp.loc["cocktail"],
        color=number_colors[group],
        marker=number_markers[group],
    )
    ax2.plot(
        df_tmp.loc["broadcast"].index,
        df_tmp.loc["broadcast"],
        color=number_colors[group],
        marker=number_markers[group],
        linestyle="--",
    )

ax2.legend(
    [Line2D([], [], color="k"), Line2D([], [], color="k", linestyle="--")],
    ["MAS-C", "MAS-B"],
)

ax2.set_ylabel("Number of iterations")
ax2.set_xlabel("Number of jobs")
ax2.set_ylim(0, 32)

plt.savefig(
    FIGS_DIR / "makespan_by_jobs_modules_and_number_of_iterations.pdf",
    bbox_inches="tight",
    pad_inches=0,
)
plt.show()

In [ ]:
from matplotlib.text import Text

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 2.1), dpi=250, sharey=True)
plt.subplots_adjust(wspace=0.05)
ax2.axhline(10 * 60, color="r")
ax1.axhline(10 * 60, color="r")
for group, df in df_generic.query("algorithm == 'bhcs'").groupby("modules"):
    df_tmp = df.groupby(["jobs", "modular_algorithm"], as_index=False)["total_time"].mean()
    df_mod = df_tmp.query("modular_algorithm == 'cocktail'")
    group = cast(int, group)
    ax1.plot(
        df_mod["jobs"],
        df_mod["total_time"] / 1000,
        color=number_colors[group],
        marker=number_markers[group],
    )
    df_mon = df_tmp.query("modular_algorithm == 'broadcast'")
    ax2.plot(
        df_mon["jobs"],
        df_mon["total_time"] / 1000,
        color=number_colors[group],
        marker=number_markers[group],
    )

ax2.text(-90, 10 * 60 - 50, "Time limit")
ax1.set_xlabel("Number of jobs [MAS-C]")
ax2.set_xlabel("Number of jobs [MAS-B]")
ax1.set_ylabel("Execution time [s]")
l = fig.legend(
    # [ Line2D([], [], color='none') ] + [
    [
        Line2D([0], [0], color=number_colors[i], marker=number_markers[i], markersize=5.0)
        for i in range(2, 11)
    ],
    [f"{i}" for i in range(2, 11)],
    # ["Modules"] + [f'{i}' for i in range(2, 11)],
    loc="upper left",
    bbox_to_anchor=(0.116, 1.03),
    title="Modules",
    ncol=9,
    columnspacing=0.5,
)

# From https://stackoverflow.com/a/53329898/4005637
c = l.get_children()[0]
title = c.get_children()[0]
hpack = c.get_children()[1]
c._children = [hpack]
hpack._children = [title] + hpack.get_children()

# l.get_title().set_position((-630, -45))
plt.savefig(FIGS_DIR / "execution_time_by_jobs_modules_c_b.pdf", bbox_inches="tight", pad_inches=0)
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 2.1), dpi=250, sharey=True)
plt.subplots_adjust(wspace=0.05)

for group, df in df_generic.query("algorithm == 'bhcs'").groupby("modules"):
    group = cast(int, group)
    df_tmp = df.groupby(["modular_algorithm", "jobs"])["solved"].mean()
    ax1.plot(
        df_tmp.loc["cocktail"].index,
        df_tmp.loc["cocktail"],
        color=number_colors[group],
        marker=number_markers[group],
    )
    ax2.plot(
        df_tmp.loc["broadcast"].index,
        df_tmp["broadcast"],
        color=number_colors[group],
        marker=number_markers[group],
        # linestyle="--",
    )

l = ax1.legend(
    [
        Line2D([0], [0], color=number_colors[i], marker=number_markers[i], markersize=5.0)
        for i in range(2, 11)
    ],
    [f"{i}" for i in range(2, 11)],
    loc="upper left",
    bbox_to_anchor=(-0.023, 1.2),
    title="Modules",
    ncol=9,
    columnspacing=1.2,
    handletextpad=0.8,
    handlelength=2,
)

# From https://stackoverflow.com/a/53329898/4005637
c = l.get_children()[0]
title = c.get_children()[0]
hpack = c.get_children()[1]
c._children = [hpack]
hpack._children = [title] + hpack.get_children()

ax1.set_ylabel("Schedulability ratio")
ax1.set_xlabel("Number of jobs [MAS-C]")
ax2.set_xlabel("Number of jobs [MAS-B]")
plt.savefig(
    FIGS_DIR / "schedulability_ratio_by_jobs_modules.pdf", bbox_inches="tight", pad_inches=0
)
plt.show()

In [ ]:
df_optimal = df_computational.query("modular_algorithm == 'constraint' & optimal")[
    ["group", "original_path", "file_id", "makespan"]
].copy()
df_optimal["have_optimal"] = True

df_optimal = df_computational.merge(
    df_optimal, on=["group", "original_path", "file_id"], how="left", suffixes=(None, "_optimal")
).query("have_optimal == True & algorithm != 'simple'")

df_optimal["optimality_gap"] = (
    df_optimal["makespan"] - df_optimal["makespan_optimal"]
) / np.maximum(1e-10, df_optimal["makespan"])

df_optimal.loc[np.isclose(df_optimal["makespan"], df_optimal["makespan_optimal"]), "optimal"] = True

df_o = df_optimal.groupby(["modular_algorithm", "time_limit"])["optimal"].agg(["mean", "sum"])
df_o["mean"] *= 100
df_o = df_o.rename(columns={"mean": "%"})
display(df_o.style.format("{:.1f}", ["%"]))

In [ ]:
# Draw a linear plot of the number of optimal solutions found by MON depending on the
# number of jobs and modules
plt.figure(figsize=(1.9, 2.1), dpi=250)

df_selected = df_optimal.query("modular_algorithm == 'constraint'")
df_tmp = df_selected.value_counts("modules")
sns.set_theme(context="paper", style="whitegrid")
plt.plot(
    df_tmp.index,
    df_tmp,
    color="tab:blue",
    marker=".",
)

# plt.ylim(0, 11)

plt.xlim(1.5, 10.5)
plt.xticks([2, 4, 6, 8, 10])
plt.tick_params(axis="y", pad=-2)

plt.ylabel("# optimally solved")
plt.xlabel("# of modules")
plt.savefig(FIGS_DIR / "computational_mon_optimal.pdf", bbox_inches="tight", pad_inches=0)

In [ ]:
# Draw box plot comparing the optimality gap
plt.figure(figsize=(1.6, 2.3), dpi=250)
sns.set_theme(context="paper", style="whitegrid")

df_selected = df_optimal.query("modular_algorithm != 'constraint' & time_limit==3600")
g = sns.boxplot(
    data=df_selected,
    x="algorithm_name",
    y="optimality_gap",
    hue="algorithm_name",
    hue_order=["MAS-B", "MAS-C"],
    palette=colors,
    legend=False,
    whis=(5, 95),
    showfliers=False,
    order=["MAS-B", "MAS-C"],
)

hatches_lst = [hatches["MAS-B"], hatches["MAS-C"]]
for patch, hatch in zip(g.patches, hatches_lst):
    patch.set_hatch(hatch)

plt.ylim(-0.01, 0.21)
plt.ylabel("Optimality gap")
plt.xlabel("Strategy")
plt.tick_params(axis="y", pad=-2)
plt.grid(axis="x")
plt.savefig(FIGS_DIR / "computational_optimality_gap.pdf", bbox_inches="tight", pad_inches=0)
plt.show()

In [ ]:
# Draw the relative makespan using MAS with 3600 seconds as a reference

df_compared = plot.baseline_compare(df_computational, "constraint", "makespan")
df_compared = df_compared.query("algorithm != 'simple'")
display(df_compared.groupby(["modular_algorithm", "time_limit"])["makespan"].describe())

plt.figure(figsize=(1.9, 2.3), dpi=250)
sns.set_theme(context="paper", style="whitegrid")

g = sns.boxplot(
    data=df_compared,
    x="algorithm_name",
    y="makespan",
    hue="time_limit",
    hue_order=[600, 3600],
    palette=colors,
    legend=False,
    whis=(5, 95),
    showfliers=False,
    order=["MAS-B", "MAS-C"],
)

hatches_lst = [hatches[600]] * 2 + [hatches[3600]] * 2
for patch, hatch in zip(g.patches, hatches_lst):
    patch.set_hatch(hatch)

from matplotlib.patches import Patch

plt.legend(
    handles=[
        Patch(facecolor=colors[600], edgecolor="k", hatch=hatches[600], label="600"),
        Patch(facecolor=colors[3600], edgecolor="k", hatch=hatches[3600], label="3600"),
    ],
    loc="upper right",
    bbox_to_anchor=(1.65, 1.04),
    title="Time limit [s]",
)

plt.ylabel("Relative makespan")
plt.xlabel("Strategy")
plt.tick_params(axis="y", pad=-2)
plt.grid(axis="x")
plt.savefig(FIGS_DIR / "computational_makespan.pdf", bbox_inches="tight", pad_inches=0)
plt.show()

In [ ]:
# Plot boxplot on the number of iterations when comparing the algorithms and the time limits

plt.figure(figsize=(2.2, 2.3), dpi=250)
sns.set_theme(context="paper", style="whitegrid")

df_selected = df_computational.query("modular_algorithm != 'constraint' & algorithm != 'simple'")
g = sns.boxplot(
    data=df_selected,
    x="algorithm_name",
    y="iterations",
    hue="time_limit",
    hue_order=[600, 3600],
    palette=colors,
    legend=False,
    whis=(5, 95),
    showfliers=False,
    order=["MAS-B", "MAS-C"],
)

hatches_lst = [hatches[600]] * 2 + [hatches[3600]] * 2
for patch, hatch in zip(g.patches, hatches_lst):
    patch.set_hatch(hatch)

from matplotlib.patches import Patch

plt.legend(
    handles=[
        Patch(facecolor=colors[600], edgecolor="k", hatch=hatches[600], label="600"),
        Patch(facecolor=colors[3600], edgecolor="k", hatch=hatches[3600], label="3600"),
    ],
    loc="upper right",
    bbox_to_anchor=(1.562, 1.04),
    title="Time limit [s]",
)

plt.ylim(0, None)
plt.ylabel("# iterations")
plt.xlabel("Strategy")
plt.tick_params(axis="y", pad=-2)
plt.grid(axis="x")
plt.savefig(FIGS_DIR / "computational_iterations.pdf", bbox_inches="tight", pad_inches=0)
plt.show()

display(df_selected.groupby(["modular_algorithm", "time_limit"])["solved"].mean().to_frame())

In [ ]:
df_compared = plot.baseline_compare(df_mixed_100, "constraint", "makespan")
display(
    df_compared.groupby(["modular_algorithm", "algorithm", "time_limit"])["makespan"].describe()
)

plt.figure(figsize=(1.9, 2.3), dpi=250)
sns.set_theme(context="paper", style="whitegrid")

order = ["MAS-BHCS", "MAS-MNEH", "MAS-INTL"]
g = sns.boxplot(
    data=df_compared,
    x="algorithm_name",
    y="makespan",
    hue="algorithm_name",
    hue_order=order,
    palette=colors,
    legend=False,
    whis=(5, 95),
    showfliers=False,
    order=order,
)

for patch, hatch in zip(g.patches, order):
    patch.set_hatch(hatches[hatch])


plt.ylabel("Relative makespan")
plt.xlabel("Strategy")
plt.xticks([0, 1, 2], ["MAS\nBHCS", "MAS\nMNEH", "MAS\nINTL"])
plt.tick_params(axis="y", pad=-2)
plt.savefig(FIGS_DIR / "mixed_100_makespan.pdf", bbox_inches="tight", pad_inches=0)
plt.show()

In [ ]:
df_compared = plot.baseline_compare(df_mixed_60, "constraint", "makespan")
display(
    df_compared.groupby(["modular_algorithm", "algorithm", "time_limit"])["makespan"].describe()
)

plt.figure(figsize=(1.9, 2.3), dpi=250)
sns.set_theme(context="paper", style="whitegrid")

order = ["MAS-BHCS", "MAS-MNEH", "MAS-INTL"]
g = sns.boxplot(
    data=df_compared,
    x="algorithm_name",
    y="makespan",
    hue="algorithm_name",
    hue_order=["MAS-BHCS", "MAS-MNEH", "MAS-INTL"],
    palette=colors,
    legend=False,
    whis=(5, 95),
    showfliers=False,
    order=order,
)

for patch, hatch in zip(g.patches, order):
    patch.set_hatch(hatches[hatch])


plt.ylabel("Relative makespan")
plt.xlabel("Strategy")
plt.xticks([0, 1, 2], ["MAS\nBHCS", "MAS\nMNEH", "MAS\nINTL"])
plt.tick_params(axis="y", pad=-2)
plt.savefig(FIGS_DIR / "mixed_60_makespan.pdf", bbox_inches="tight", pad_inches=0)
plt.show()

In [ ]:
df_tmp = (
    df_mixed_100.query("modular_algorithm != 'constraint'")
    .groupby(["algorithm_name"])["solved"]
    .mean()
    * 100
).to_frame()
display(df_tmp)

plt.figure(figsize=(1.9, 2.3), dpi=250)
sns.set_theme(context="paper", style="whitegrid")

order = ["MAS-BHCS", "MAS-MNEH", "MAS-INTL"]
g = sns.barplot(
    data=df_tmp,
    x="algorithm_name",
    y="solved",
    hue="algorithm_name",
    hue_order=order,
    palette=colors,
    legend=False,
    edgecolor=".1",
    order=order,
)

hatches_lst = ["MAS-BHCS", "MAS-MNEH", "MAS-INTL"]
for patch, hatch in zip(g.patches, hatches_lst):
    patch.set_hatch(hatches[hatch])

plt.xticks([0, 1, 2], ["MAS\nBHCS", "MAS\nMNEH", "MAS\nINTL"])
plt.ylabel("Schedulability ratio [%]")
plt.xlabel("Local algorithm")
plt.savefig(FIGS_DIR / "mixed_100_schedulability.pdf", bbox_inches="tight", pad_inches=0)
plt.show()


In [ ]:
df_tmp = (
    df_mixed_60.query("modular_algorithm != 'constraint'")
    .groupby(["algorithm_name"])["solved"]
    .mean()
    * 100
).to_frame()
display(df_tmp)

plt.figure(figsize=(1.9, 2.3), dpi=250)
sns.set_theme(context="paper", style="whitegrid")

order = ["MAS-BHCS", "MAS-MNEH", "MAS-INTL"]
g = sns.barplot(
    data=df_tmp,
    x="algorithm_name",
    y="solved",
    hue="algorithm_name",
    hue_order=order,
    palette=colors,
    legend=False,
    edgecolor=".1",
    order=order,
)

hatches_lst = ["MAS-BHCS", "MAS-MNEH", "MAS-INTL"]
for patch, hatch in zip(g.patches, hatches_lst):
    patch.set_hatch(hatches[hatch])

plt.xticks([0, 1, 2], ["MAS\nBHCS", "MAS\nMNEH", "MAS\nINTL"])
plt.ylabel("Schedulability ratio [%]")
plt.xlabel("Local algorithm")
plt.ylim(0, 100)
plt.savefig(FIGS_DIR / "mixed_60_schedulability.pdf", bbox_inches="tight", pad_inches=0)
plt.show()


In [ ]:
df_tmp = df_mixed_100.query("modular_algorithm != 'constraint'").copy()
df_tmp["total_time"] /= 1000
display(
    df_tmp.groupby(["algorithm_name", "algorithm"])["total_time"].describe()
)

plt.figure(figsize=(1.9, 2.3), dpi=250)
sns.set_theme(context="paper", style="whitegrid")

order = ["MAS-BHCS", "MAS-MNEH", "MAS-INTL"]
g = sns.boxplot(
    data=df_tmp,
    x="algorithm_name",
    y="total_time",
    hue="algorithm_name",
    hue_order=order,
    palette=colors,
    legend=False,
    whis=(5, 95),
    showfliers=False,
    order=order,
)

for patch, hatch in zip(g.patches, order):
    patch.set_hatch(hatches[hatch])


plt.ylabel("Execution time [s]")
plt.xlabel("Strategy")
plt.xticks([0, 1, 2], ["MAS\nBHCS", "MAS\nMNEH", "MAS\nINTL"])
plt.tick_params(axis="y", pad=-2)
plt.savefig(FIGS_DIR / "mixed_100_execution_time.pdf", bbox_inches="tight", pad_inches=0)
plt.show()

In [ ]:
df_tmp = df_mixed_60.query("modular_algorithm != 'constraint'").copy()
df_tmp["total_time"] /= 1000
display(
    df_tmp.groupby(["algorithm_name", "algorithm"])["total_time"].describe()
)

plt.figure(figsize=(1.9, 2.3), dpi=250)
sns.set_theme(context="paper", style="whitegrid")

order = ["MAS-BHCS", "MAS-MNEH", "MAS-INTL"]
g = sns.boxplot(
    data=df_tmp,
    x="algorithm_name",
    y="total_time",
    hue="algorithm_name",
    hue_order=order,
    palette=colors,
    legend=False,
    whis=(5, 95),
    showfliers=False,
    order=order,
)

for patch, hatch in zip(g.patches, order):
    patch.set_hatch(hatches[hatch])


plt.ylabel("Execution time [s]")
plt.xlabel("Strategy")
plt.xticks([0, 1, 2], ["MAS\nBHCS", "MAS\nMNEH", "MAS\nINTL"])
plt.tick_params(axis="y", pad=-2)
plt.savefig(FIGS_DIR / "mixed_60_execution_time.pdf", bbox_inches="tight", pad_inches=0)
plt.show()